# Install geotable, GDAL

```
$ brew install gdal libkml
$ pip install GDAL geotable
```

# Data download

[Map](https://www.google.com/maps/d/u/0/viewer?mid=1hQhJDhsE87Iu9BJOln-EnveGbow&ll=35.07345181763887%2C138.8560247350858&z=12)

[KML Download](https://www.google.com/maps/d/u/0/kml?forcekml=1&mid=1hQhJDhsE87Iu9BJOln-EnveGbow&lid=Cs8qfjzbvoQ)

In [92]:
!cp ~/Downloads/沼津まちあるきスタンプ\　設置店舗.kml ~/code/xkeeja/lovelive-machiaruki/data/machiaruki.kml

# Geotable

In [79]:
import os
import geotable
import requests

In [86]:
data_kml = os.path.join(os.path.abspath('..'), 'data/machiaruki.kml')
data_kml

'/Users/rkb/code/xkeeja/lovelive-machiaruki/data/machiaruki.kml'

In [96]:
t = geotable.load(data_kml)
t

,Name,Description,geometry_object,geometry_layer,geometry_proj4
0,ゲーマーズ沼津店,"<img src=""https://doc-14-4g-mymaps.googleuserc...",POINT Z (138.856807 35.10157 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
1,安田屋旅館,"<img src=""https://doc-08-4g-mymaps.googleuserc...",POINT Z (138.897573 35.020424 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
2,和洋菓子松月,"<img src=""https://doc-14-4g-mymaps.googleuserc...",POINT Z (138.897925 35.022259 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
3,SUN!SUN!サンシャインCAFE,"<img src=""https://doc-0s-4g-mymaps.googleuserc...",POINT Z (138.8597404 35.1019932 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
4,オーシャン ビュー フジミ,"<img src=""https://doc-0o-4g-mymaps.googleuserc...",POINT Z (138.7858531 35.0249211 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
...,...,...,...,...,...
112,グランマ シーサイド店,"<img src=""https://doc-0s-4g-mymaps.googleuserc...",POINT Z (138.8860146 35.0481496 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
113,テレビのイマイ,"<img src=""https://doc-0s-4g-mymaps.googleuserc...",POINT Z (138.8578183 35.0986286 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
114,戸田観光協会,"<img src=""https://doc-0g-4g-mymaps.googleuserc...",POINT Z (138.7772204 34.9729703 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
115,モスバーガーららぽーと沼津店,"<img src=""https://doc-14-4g-mymaps.googleuserc...",POINT Z (138.8419064 35.1204196 0),沼津まちあるきスタンプ 設置店舗,+proj=longlat +datum=WGS84 +no_defs
